In [61]:
import re
import nltk
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

In [62]:
data = pd.read_csv(r'C:/usf/Data Science Programming/project/Combined_News_DJIA.csv')
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

In [63]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [64]:
trainheadlines[1]

'b\'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?\' b\'Bush puts foot down on Georgian conflict\' b"Jewish Georgian minister: Thanks to Israeli training, we\'re fending off Russia " b\'Georgian army flees in disarray as Russians advance - Gori abandoned to Russia without a shot fired\' b"Olympic opening ceremony fireworks \'faked\'" b\'What were the Mossad with fraudulent New Zealand Passports doing in Iraq?\' b\'Russia angered by Israeli military sale to Georgia\' b\'An American citizen living in S.Ossetia blames U.S. and Georgian leaders for the genocide of innocent people\' b\'Welcome To World War IV! Now In High Definition!\' b"Georgia\'s move, a mistake of monumental proportions " b\'Russia presses deeper into Georgia; U.S. says regime change is goal\' b\'Abhinav Bindra wins first ever Individual Olympic Gold Medal for India\' b\' U.S. ship heads for Arctic to define territory\' b\'Drivers in a Jerusalem taxi station threaten to quit ra

In [65]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

(1611, 31675)


Logic Regression

In [66]:
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])

C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [68]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(testheadlines)
preds1 = basicmodel.predict(basictest)
acc1=accuracy_score(test['Label'], preds1)
print('Logic Regression 1 accuracy: ',acc1 )

Logic Regression 1 accuracy:  0.42592592592592593


In [69]:
basicwords = basicvectorizer.get_feature_names()
basiccoeffs = basicmodel.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
print(coeffdf.head(3))
print(coeffdf.tail(3))

          Word  Coefficient
19419  nigeria     0.497924
25261     self     0.452526
29286       tv     0.428011
            Word  Coefficient
25433        sex    -0.494555
24754  sanctions    -0.549725
24542        run    -0.587794


Logic Regression 2

In [70]:
#The counts of phrases with two connected words(exclude words which are too common like "a" ,"an" ,"the" and words too uncommon of which counts are too small )

#We delete phrases of which frequency lower than 0.03 or higher than 0.97

advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.97, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [73]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds2 = advancedmodel.predict(advancedtest)
acc2=accuracy_score(test['Label'], preds2)

print('Logic Regression 2 accuracy: ', acc2)

Logic Regression 2 accuracy:  0.5714285714285714


In [74]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])

Logic Regression 3

In [75]:
#The counts of phrases with three connected words(exclude words which are too common like "a" ,"an" ,"the" and words too uncommon of which counts are too small )

advancedvectorizer = TfidfVectorizer( min_df=0.0039, max_df=0.1, max_features = 200000, ngram_range = (3, 3))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [76]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

In [78]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,3:28]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds3 = advancedmodel.predict(advancedtest)
acc3 = accuracy_score(test['Label'], preds3)
print('Logic Regression 3 accuracy: ', acc3)

Logic Regression 3 accuracy:  0.5582010582010583


In [79]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
print(advcoeffdf.head(3))
print(advcoeffdf.tail(3))

             Words  Coefficient
2075     to the us     1.076893
686   in china the     0.886762
1775  the right to     0.853562
                      Words  Coefficient
1317  phone hacking scandal    -0.943155
152        around the world    -0.972078
779          in the country    -1.154834


Naive Bayes

In [80]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.7, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [82]:
advancedmodel = MultinomialNB(alpha=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,3:28]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds4 = advancedmodel.predict(advancedtest)
acc4=accuracy_score(test['Label'], preds4)
print('NBayes 1 accuracy: ', acc4)

NBayes 1 accuracy:  0.5132275132275133


In [83]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
print(advcoeffdf.head(3))
print(advcoeffdf.tail(3))

      Words  Coefficient
234  israel    -5.180422
497     was    -5.252936
94    china    -5.312797
      Words  Coefficient
426   story    -6.952040
290  monday    -6.970522
268    lost    -6.970642


NBayes 2

In [84]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [86]:
advancedmodel = MultinomialNB(alpha=0.0001)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds5 = advancedmodel.predict(advancedtest)
acc5 = accuracy_score(test['Label'], preds5)
print('NBayes 2 accuracy: ', acc5)

NBayes 2 accuracy:  0.5423280423280423


In [87]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
print(advcoeffdf.head(3))
print(advcoeffdf.tail(3))

        Words  Coefficient
49   at least    -5.495760
185  in china    -5.499764
247     it is    -5.543152
           Words  Coefficient
172        if it    -7.212155
84   children in    -7.218639
383     said the    -7.323029


Random Forest

In [88]:
advancedvectorizer = TfidfVectorizer( min_df=0.01, max_df=0.99, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [89]:
advancedmodel = RandomForestClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds6 = advancedmodel.predict(advancedtest)
acc6 = accuracy_score(test['Label'], preds6)
print('RF 1 accuracy: ', acc6)

C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF 1 accuracy:  0.5052910052910053


RF2

In [91]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
advancedmodel = RandomForestClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds7 = advancedmodel.predict(advancedtest)
acc7 = accuracy_score(test['Label'], preds7)
print('RF 2 accuracy: ', acc7)

C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF 2 accuracy:  0.5423280423280423


Gradient Boosting Machines

In [92]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.9, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [94]:
advancedmodel = GradientBoostingClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds8 = advancedmodel.predict(advancedtest.toarray())
acc8 = accuracy_score(test['Label'], preds8)
print('GBM 1 accuracy: ', acc8)

GBM 1 accuracy:  0.4497354497354497


GBM 2

In [95]:
advancedvectorizer = TfidfVectorizer( min_df=0.02, max_df=0.175, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [97]:
advancedmodel = GradientBoostingClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,3:28]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds9 = advancedmodel.predict(advancedtest.toarray())
acc9 = accuracy_score(test['Label'], preds9)
print('GBM 2 accuracy: ', acc9)

GBM 2 accuracy:  0.5291005291005291


Stochastic Gradient Descent Classifier

In [98]:
advancedvectorizer = TfidfVectorizer( min_df=0.2, max_df=0.8, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [99]:
advancedmodel = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds10 = advancedmodel.predict(advancedtest.toarray())
acc10 = accuracy_score(test['Label'], preds10)
print('SGDClassifier 1: ', acc10)

C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDClassifier 1:  0.5052910052910053


SGDClassifier 2

In [100]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [101]:
advancedmodel = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds11 = advancedmodel.predict(advancedtest.toarray())
acc11 = accuracy_score(test['Label'], preds11)
print('SGDClassifier 2: ', acc11)

C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDClassifier 2:  0.5423280423280423


Naive Bayes SVM

In [102]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):

    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

        #p_c /= np.linalg.norm(p_c, ord=1)
        #ratios[c] = np.log(p_c / (1 - p_c))


def f1_class(pred, truth, class_val):
    n = len(truth)

    truth_class = 0
    pred_class = 0
    tp = 0

    for ii in range(0, n):
        if truth[ii] == class_val:
            truth_class += 1
            if truth[ii] == pred[ii]:
                tp += 1
                pred_class += 1
                continue;
        if pred[ii] == class_val:
            pred_class += 1

    precision = tp / float(pred_class)
    recall = tp / float(truth_class)

    return (2.0 * precision * recall) / (precision + recall)


def semeval_senti_f1(pred, truth, pos=2, neg=0): 

    f1_pos = f1_class(pred, truth, pos)
    f1_neg = f1_class(pred, truth, neg)

    return (f1_pos + f1_neg) / 2.0;


def main(train_file, test_file, ngram=(1, 3)):
    print('loading...')
    train = pd.read_csv(train_file, delimiter='\t', encoding='utf-8', header=0,
                        names=['text', 'label'])

    # to shuffle:
    #train.iloc[np.random.permutation(len(df))]

    test = pd.read_csv(test_file, delimiter='\t', encoding='utf-8', header=0,
                        names=['text', 'label'])

    print('vectorizing...')
    vect = CountVectorizer()
    classifier = NBSVM()

    # create pipeline
    clf = Pipeline([('vect', vect), ('nbsvm', classifier)])
    params = {
        'vect__token_pattern': r"\S+",
        'vect__ngram_range': ngram, 
        'vect__binary': True
    }
    clf.set_params(**params)

    #X_train = vect.fit_transform(train['text'])
    #X_test = vect.transform(test['text'])

    print('fitting...')
    clf.fit(train['text'], train['label'])

    print('classifying...')
    pred = clf.predict(test['text'])
   
    print('testing...')
    acc = accuracy_score(test['label'], pred)
    f1 = semeval_senti_f1(pred, test['label'])
    print('NBSVM: acc=%f, f1=%f' % (acc, f1))

NBSVM 1

In [103]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.8, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1611, 535)


In [105]:
advancedmodel = NBSVM(C=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds12 = advancedmodel.predict(advancedtest)
acc12 = accuracy_score(test['Label'], preds12)
print('NBSVM 1: ', acc12)

NBSVM 1:  0.47883597883597884


NBSVM 2

In [106]:
advancedvectorizer = TfidfVectorizer( min_df=0.031, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1611, 613)


In [108]:
advancedmodel = NBSVM(C=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds13 = advancedmodel.predict(advancedtest)
acc13 = accuracy_score(test['Label'], preds13)
print('NBSVM 2: ', acc13)

NBSVM 2:  0.5952380952380952


Deep Learning

In [109]:
#MLP
batch_size = 32
nb_classes = 2
advancedvectorizer = TfidfVectorizer( min_df=0.04, max_df=0.3, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
print(advancedtrain.shape)

(1611, 401)


In [110]:
X_train = advancedtrain.toarray()
X_test = advancedtest.toarray()

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(train["Label"])
y_test = np.array(test["Label"])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.mean(X_train)
X_train -= mean
X_test -= mean

input_dim = X_train.shape[1]

# Here's a Deep Dumb MLP (DDMLP)
model = Sequential()
model.add(Dense(256, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print("Training...")
model.fit(X_train, Y_train, nb_epoch=2, batch_size=16, validation_split=0.15)

print("Generating test predictions...")
preds14 = model.predict_classes(X_test, verbose=0)
acc14 = accuracy_score(test["Label"], preds14)

X_train shape: (1611, 401)
X_test shape: (378, 401)
Training...


C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\ipykernel_launcher.py:39: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1369 samples, validate on 242 samples
Epoch 1/2
1369/1369 [==============================] - 1s 533us/step - loss: 0.6937 - val_loss: 0.6812
Epoch 2/2
1369/1369 [==============================] - 0s 184us/step - loss: 0.6660 - val_loss: 0.6751
Generating test predictions...


In [112]:
print('prediction accuracy: ', acc14)

prediction accuracy:  0.5476190476190477


In [113]:
max_features = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1
maxlen = 200
batch_size = 32
nb_classes = 2

In [114]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=max_features)
tokenizer.fit_on_texts(trainheadlines)
sequences_train = tokenizer.texts_to_sequences(trainheadlines)
sequences_test = tokenizer.texts_to_sequences(testheadlines)


C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [115]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (1611, 200)
X_test shape: (378, 200)


In [116]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2)) 
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=3,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


print("Generating test predictions...")
preds15 = model.predict_classes(X_test, verbose=0)
acc15 = accuracy_score(test['Label'], preds15)

Build model...


C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  after removing the cwd from sys.path.


Train...


C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 1611 samples, validate on 378 samples
Epoch 1/3
1611/1611 [==============================] - 16s 10ms/step - loss: 0.6911 - acc: 0.5400 - val_loss: 0.6943 - val_acc: 0.5079
Epoch 2/3
1611/1611 [==============================] - 14s 9ms/step - loss: 0.6615 - acc: 0.6418 - val_loss: 0.6841 - val_acc: 0.5741
Epoch 3/3
378/378 [==============================] - 1s 2ms/step
Test score: 0.8248221029049505
Test accuracy: 0.5661375661375662
Generating test predictions...


In [118]:
print('prediction accuracy: ', acc15)

prediction accuracy:  0.5661375661375662


CNN

In [119]:
nb_filter = 120
filter_length = 2
hidden_dims = 120
nb_epoch = 2

In [120]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))

def max_1d(X):
    return K.max(X, axis=1)

model.add(Lambda(max_1d, output_shape=(nb_filter,)))
model.add(Dense(hidden_dims)) 
model.add(Dropout(0.2)) 
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=120, kernel_size=2, strides=1, padding="valid")`
  # Remove the CWD from sys.path while we load stuff.


In [121]:
print('Train...')
model.fit(X_train, Y_train, batch_size=32, nb_epoch=1,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


print("Generating test predictions...")
preds16 = model.predict_classes(X_test, verbose=0)
acc16 = accuracy_score(test['Label'], preds16)

Train...


C:\Users\bibin\Anaconda2\envs\tf_env\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 1611 samples, validate on 378 samples
Epoch 1/1
378/378 [==============================] - 0s 393us/step
Test score: 0.6943733509255465
Test accuracy: 0.5079365098287189
Generating test predictions...


In [122]:
print('prediction accuracy: ', acc16)

prediction accuracy:  0.5079365079365079


In [123]:
print('Logic Regression 1 accuracy: ',acc1 )
print('Logic Regression 2 accuracy: ', acc2)
print('Logic Regression 3 accuracy: ', acc3)
print('NBayes 1 accuracy: ', acc4)
print('NBayes 2 accuracy: ', acc5)
print('RF 1 accuracy: ', acc6)
print('RF 2 accuracy: ', acc7)
print('GBM 1 accuracy: ', acc8)
print('GBM 2 accuracy: ', acc9)
print('SGDClassifier 1: ', acc10)
print('SGDClassifier 2: ', acc11)
print('NBSVM 1: ', acc12)
print('NBSVM 2: ', acc13)
print('DL1 prediction accuracy: ', acc14)
print('DL2 prediction accuracy: ', acc15)
print('CNN prediction accuracy: ', acc16)

Logic Regression 1 accuracy:  0.42592592592592593
Logic Regression 2 accuracy:  0.5714285714285714
Logic Regression 3 accuracy:  0.5582010582010583
NBayes 1 accuracy:  0.5132275132275133
NBayes 2 accuracy:  0.5423280423280423
RF 1 accuracy:  0.5052910052910053
RF 2 accuracy:  0.5423280423280423
GBM 1 accuracy:  0.4497354497354497
GBM 2 accuracy:  0.5291005291005291
SGDClassifier 1:  0.5052910052910053
SGDClassifier 2:  0.5423280423280423
NBSVM 1:  0.47883597883597884
NBSVM 2:  0.5952380952380952
DL1 prediction accuracy:  0.5476190476190477
DL2 prediction accuracy:  0.5661375661375662
CNN prediction accuracy:  0.5079365079365079
